### init the engine

In [17]:
from sqlalchemy import create_engine
engine = create_engine('postgresql://mahtin@localhost:5432/mahtin', echo=True)

### declare a schema

In [18]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [19]:
from sqlalchemy import Column, Integer, String
class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(80))
    fullname = Column(String(80))
    nickname = Column(String(50))
    
    def __repr__(self):
        return f"<User(name={self.name}, fullname={self.fullname}, nickname={self.nickname})"

### create the table(s)

In [20]:
Base.metadata.create_all(engine)

2019-07-30 14:15:08,170 INFO sqlalchemy.engine.base.Engine select version()
2019-07-30 14:15:08,171 INFO sqlalchemy.engine.base.Engine {}
2019-07-30 14:15:08,173 INFO sqlalchemy.engine.base.Engine select current_schema()
2019-07-30 14:15:08,173 INFO sqlalchemy.engine.base.Engine {}
2019-07-30 14:15:08,175 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-07-30 14:15:08,175 INFO sqlalchemy.engine.base.Engine {}
2019-07-30 14:15:08,177 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-07-30 14:15:08,177 INFO sqlalchemy.engine.base.Engine {}
2019-07-30 14:15:08,178 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2019-07-30 14:15:08,179 INFO sqlalchemy.engine.base.Engine {}
2019-07-30 14:15:08,180 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
20

### creating a session class

In [21]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine) # the bind can be defined later over Session.configure(bind=engine)
session = Session()

### initiate a session and add objects

In [22]:
ed_user = User(name='ed', fullname='Ed Jones', nickname='Eddy')
ed_user.fullname
session.add(ed_user) # session is pending
our_user = session.query(User).filter_by(name='ed').all()[0]
our_user
our_user2 = session.query(User).filter_by(name='ed').first()
our_user2

'Ed Jones'

2019-07-30 14:15:18,016 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-30 14:15:18,018 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (%(name)s, %(fullname)s, %(nickname)s) RETURNING users.id
2019-07-30 14:15:18,018 INFO sqlalchemy.engine.base.Engine {'name': 'ed', 'fullname': 'Ed Jones', 'nickname': 'Eddy'}
2019-07-30 14:15:18,021 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = %(name_1)s
2019-07-30 14:15:18,022 INFO sqlalchemy.engine.base.Engine {'name_1': 'ed'}


<User(name=ed, fullname=Ed Jones, nickname=Eddy)

2019-07-30 14:15:18,027 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = %(name_1)s 
 LIMIT %(param_1)s
2019-07-30 14:15:18,027 INFO sqlalchemy.engine.base.Engine {'name_1': 'ed', 'param_1': 1}


<User(name=ed, fullname=Ed Jones, nickname=Eddy)

In [23]:
session.add_all([
...     User(name='wendy', fullname='Wendy Williams', nickname='windy'),
...     User(name='mary', fullname='Mary Contrary', nickname='mary'),
...     User(name='fred', fullname='Fred Flintstone', nickname='freddy')])

In [24]:
ed_user.nickname = 'Eddo'

In [28]:
session.query(User).first()

2019-07-30 14:19:08,445 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
 LIMIT %(param_1)s
2019-07-30 14:19:08,446 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


<User(name=ed, fullname=Ed Jones, nickname=Eddy)

In [10]:
session.commit()

In [15]:
for data in session.query(User, User.id):
    print(data)

(<User(name=ed, fullname=Ed Jones, nickname=Eddy), 1)
(<User(name=wendy, fullname=Wendy Williams, nickname=windy), 3)
(<User(name=mary, fullname=Mary Contrary, nickname=mary), 4)
(<User(name=fred, fullname=Fred Flintstone, nickname=freddy), 5)
(<User(name=jack, fullname=Jack Bean, nickname=gjffdd), 6)
(<User(name=ed, fullname=Ed Jones, nickname=Eddo), 14)
(<User(name=wendy, fullname=Wendy Williams, nickname=windy), 15)
(<User(name=mary, fullname=Mary Contrary, nickname=mary), 16)
(<User(name=fred, fullname=Fred Flintstone, nickname=freddy), 17)
(<User(name=jack, fullname=Jack Bean, nickname=gjffdd), 18)
(<User(name=ed, fullname=Ed Jones, nickname=Eddo), 19)
(<User(name=wendy, fullname=Wendy Williams, nickname=windy), 20)
(<User(name=mary, fullname=Mary Contrary, nickname=mary), 21)
(<User(name=fred, fullname=Fred Flintstone, nickname=freddy), 22)
(<User(name=ed, fullname=Ed Jones, nickname=Eddo), 23)
(<User(name=wendy, fullname=Wendy Williams, nickname=windy), 24)
(<User(name=mary, ful

## Querying

+ query returns named tuples (after attributes or class in query) for every query object

In [31]:
for tuple in session.query(User, User.fullname).order_by(User.id):
    print(tuple.User.id, tuple.fullname)

2019-07-30 14:33:59,898 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users ORDER BY users.id
2019-07-30 14:33:59,899 INFO sqlalchemy.engine.base.Engine {}
1 Ed Jones
3 Wendy Williams
4 Mary Contrary
5 Fred Flintstone
6 Jack Bean
14 Ed Jones
15 Wendy Williams
16 Mary Contrary
17 Fred Flintstone
18 Jack Bean
19 Ed Jones
20 Wendy Williams
21 Mary Contrary
22 Fred Flintstone
23 Ed Jones
24 Wendy Williams
25 Mary Contrary
26 Fred Flintstone
27 Jack Bean
28 Ed Jones
29 Wendy Williams
30 Mary Contrary
31 Fred Flintstone
32 Ed Jones
33 Wendy Williams
34 Mary Contrary
35 Fred Flintstone


+ label your resulting tuple name using label and aliased

In [32]:
from sqlalchemy.orm import aliased
alUser = aliased(User, name='alUser')
for tuple in session.query(alUser, User.name.label('the_name'))[1:3]:
    print(tuple.alUser.name, tuple.the_name)

2019-07-30 14:34:07,032 INFO sqlalchemy.engine.base.Engine SELECT "alUser".id AS "alUser_id", "alUser".name AS "alUser_name", "alUser".fullname AS "alUser_fullname", "alUser".nickname AS "alUser_nickname", users.name AS the_name 
FROM users AS "alUser", users 
 LIMIT %(param_1)s OFFSET %(param_2)s
2019-07-30 14:34:07,033 INFO sqlalchemy.engine.base.Engine {'param_1': 2, 'param_2': 1}
ed wendy
ed mary


### filter and filter_by
* filter_by gives simple attribute identities
* filter allows complex expressions
* all filter commands are fully chainable

In [33]:
session.query(User).order_by(User.id).filter_by(fullname='Ed Jones').all()
session.query(User).order_by(User.id).filter_by(fullname='Ed Jones').filter_by(nickname='Eddo').all()

2019-07-30 14:34:24,042 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.fullname = %(fullname_1)s ORDER BY users.id
2019-07-30 14:34:24,043 INFO sqlalchemy.engine.base.Engine {'fullname_1': 'Ed Jones'}


[<User(name=ed, fullname=Ed Jones, nickname=Eddy),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo)]

2019-07-30 14:34:24,048 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.fullname = %(fullname_1)s AND users.nickname = %(nickname_1)s ORDER BY users.id
2019-07-30 14:34:24,049 INFO sqlalchemy.engine.base.Engine {'fullname_1': 'Ed Jones', 'nickname_1': 'Eddo'}


[<User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo)]

In [34]:
session.query(User).order_by(User.id).filter(User.fullname == 'Ed Jones').all()
session.query(User).order_by(User.id).filter(User.fullname == 'Ed Jones').filter(User.nickname == 'Eddo').all()

2019-07-30 14:37:26,093 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.fullname = %(fullname_1)s ORDER BY users.id
2019-07-30 14:37:26,094 INFO sqlalchemy.engine.base.Engine {'fullname_1': 'Ed Jones'}


[<User(name=ed, fullname=Ed Jones, nickname=Eddy),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo)]

2019-07-30 14:37:26,097 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.fullname = %(fullname_1)s AND users.nickname = %(nickname_1)s ORDER BY users.id
2019-07-30 14:37:26,098 INFO sqlalchemy.engine.base.Engine {'fullname_1': 'Ed Jones', 'nickname_1': 'Eddo'}


[<User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo)]

In [37]:
session.query(User).filter(User.fullname.like('%Will%')).all()

2019-07-30 14:40:26,811 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.fullname LIKE %(fullname_1)s
2019-07-30 14:40:26,811 INFO sqlalchemy.engine.base.Engine {'fullname_1': '%Will%'}


[<User(name=wendy, fullname=Wendy Williams, nickname=windy),
 <User(name=wendy, fullname=Wendy Williams, nickname=windy),
 <User(name=wendy, fullname=Wendy Williams, nickname=windy),
 <User(name=wendy, fullname=Wendy Williams, nickname=windy),
 <User(name=wendy, fullname=Wendy Williams, nickname=windy),
 <User(name=wendy, fullname=Wendy Williams, nickname=windy)]

In [39]:
session.query(User).filter(User.name.in_(['ed', 'wendy', 'jack'])).all()

2019-07-30 14:51:30,930 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (%(name_1)s, %(name_2)s, %(name_3)s)
2019-07-30 14:51:30,931 INFO sqlalchemy.engine.base.Engine {'name_1': 'ed', 'name_2': 'wendy', 'name_3': 'jack'}


[<User(name=ed, fullname=Ed Jones, nickname=Eddy),
 <User(name=wendy, fullname=Wendy Williams, nickname=windy),
 <User(name=jack, fullname=Jack Bean, nickname=gjffdd),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=wendy, fullname=Wendy Williams, nickname=windy),
 <User(name=jack, fullname=Jack Bean, nickname=gjffdd),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=wendy, fullname=Wendy Williams, nickname=windy),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=wendy, fullname=Wendy Williams, nickname=windy),
 <User(name=jack, fullname=Jack Bean, nickname=gjffdd),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=wendy, fullname=Wendy Williams, nickname=windy),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=wendy, fullname=Wendy Williams, nickname=windy)]

In [40]:
from sqlalchemy import and_, or_
session.query(User).filter(or_(User.name.in_(['ed', 'wendy']), User.name == 'fred', User.nickname == 'windy')).all()

2019-07-30 14:54:01,451 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (%(name_1)s, %(name_2)s) OR users.name = %(name_3)s OR users.nickname = %(nickname_1)s
2019-07-30 14:54:01,452 INFO sqlalchemy.engine.base.Engine {'name_1': 'ed', 'name_2': 'wendy', 'name_3': 'fred', 'nickname_1': 'windy'}


[<User(name=ed, fullname=Ed Jones, nickname=Eddy),
 <User(name=wendy, fullname=Wendy Williams, nickname=windy),
 <User(name=fred, fullname=Fred Flintstone, nickname=freddy),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=wendy, fullname=Wendy Williams, nickname=windy),
 <User(name=fred, fullname=Fred Flintstone, nickname=freddy),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=wendy, fullname=Wendy Williams, nickname=windy),
 <User(name=fred, fullname=Fred Flintstone, nickname=freddy),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=wendy, fullname=Wendy Williams, nickname=windy),
 <User(name=fred, fullname=Fred Flintstone, nickname=freddy),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=wendy, fullname=Wendy Williams, nickname=windy),
 <User(name=fred, fullname=Fred Flintstone, nickname=freddy),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=wendy, fullname=Wendy Williams, nickname=windy),
 <User(name=fred, 

+ you can use full SQL statements with from_statement()

In [41]:
from sqlalchemy import text
session.query(User).from_statement(text("SELECT * FROM users WHERE name=:value").params(value='ed')).first()

2019-07-30 14:54:29,443 INFO sqlalchemy.engine.base.Engine SELECT * FROM users WHERE name=%(value)s
2019-07-30 14:54:29,445 INFO sqlalchemy.engine.base.Engine {'value': 'ed'}


<User(name=ed, fullname=Ed Jones, nickname=Eddy)

### Relationships

+ ForeignKey sets the connection by restricting own keys to match (and thereby possibly relate to) keys in other tables

In [42]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

class Address(Base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    # here, addresses.user is connected to users.id
    # the table carrying the ForeignKey is the many in the relationship
    user_id = Column(Integer, ForeignKey('users.id'))
    # User.addresses is populated by Address.user where Address.user == User.id
    user = relationship("User", back_populates='addresses')
    
    def __repr__(self):
        return f"<Address(email_address={self.email_address})>"

# this is resolved as a collection (configurable but defaulting to a list)
User.addresses = relationship("Address", order_by=Address.id, back_populates="user")


In [43]:
Base.metadata.create_all(engine)

2019-07-30 14:55:44,020 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2019-07-30 14:55:44,021 INFO sqlalchemy.engine.base.Engine {'name': 'users'}
2019-07-30 14:55:44,025 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2019-07-30 14:55:44,026 INFO sqlalchemy.engine.base.Engine {'name': 'addresses'}


In [44]:
jack = User(name='jack', fullname='Jack Bean', nickname='gjffdd')

In [45]:
jack.addresses = [
...                 Address(email_address='jack@google.com'),
...                 Address(email_address='j25@yahoo.com')]

In [46]:
jack.addresses
jack.addresses[0]
jack.addresses[0].user
jack.addresses[0].user.name

[<Address(email_address=jack@google.com)>,
 <Address(email_address=j25@yahoo.com)>]

<Address(email_address=jack@google.com)>

<User(name=jack, fullname=Jack Bean, nickname=gjffdd)

'jack'

In [50]:
session.add(jack)
session.commit()

2019-07-30 14:57:16,090 INFO sqlalchemy.engine.base.Engine COMMIT


In [52]:
jack.addresses

2019-07-30 14:57:27,975 INFO sqlalchemy.engine.base.Engine SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses 
WHERE %(param_1)s = addresses.user_id ORDER BY addresses.id
2019-07-30 14:57:27,975 INFO sqlalchemy.engine.base.Engine {'param_1': 36}


[<Address(email_address=jack@google.com)>,
 <Address(email_address=j25@yahoo.com)>]

In [53]:
str(session.query(Address.user))

'SELECT users.id = addresses.user_id AS "user" \nFROM users, addresses'

### Joins
+ joins can be better in cases where filtering looks into two tables as the database does not need to load a full cross join into memory (as below)

In [54]:
session.query(User.name, Address).\
    filter(User.id==Address.user_id). \
    filter(Address.email_address=='jack@google.com').\
    all()

2019-07-30 15:02:21,400 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name, addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM users, addresses 
WHERE users.id = addresses.user_id AND addresses.email_address = %(email_address_1)s
2019-07-30 15:02:21,401 INFO sqlalchemy.engine.base.Engine {'email_address_1': 'jack@google.com'}


[('jack', <Address(email_address=jack@google.com)>),
 ('jack', <Address(email_address=jack@google.com)>),
 ('jack', <Address(email_address=jack@google.com)>),
 ('jack', <Address(email_address=jack@google.com)>)]

In [55]:
for u, a in session.query(User, Address).\
    filter(User.id==Address.user_id).\
    filter(Address.email_address=='jack@google.com').\
    all():
        print(u.name)
        print(a.email_address)

2019-07-30 15:03:36,665 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname, addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM users, addresses 
WHERE users.id = addresses.user_id AND addresses.email_address = %(email_address_1)s
2019-07-30 15:03:36,666 INFO sqlalchemy.engine.base.Engine {'email_address_1': 'jack@google.com'}
jack
jack@google.com
jack
jack@google.com
jack
jack@google.com
jack
jack@google.com


As a default, a `join()` joins two dbs on their Foreign keys

In [56]:
session.query(User).join(Address).\
    filter(Address.email_address=='jack@google.com').\
    all()

2019-07-30 15:22:21,491 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users JOIN addresses ON users.id = addresses.user_id 
WHERE addresses.email_address = %(email_address_1)s
2019-07-30 15:22:21,492 INFO sqlalchemy.engine.base.Engine {'email_address_1': 'jack@google.com'}


[<User(name=jack, fullname=Jack Bean, nickname=gjffdd),
 <User(name=jack, fullname=Jack Bean, nickname=gjffdd),
 <User(name=jack, fullname=Jack Bean, nickname=gjffdd),
 <User(name=jack, fullname=Jack Bean, nickname=gjffdd)]

Else, you can use more explicit join statements:

In [57]:
session.query(User).join(Address, User.id==Address.user_id).all()

2019-07-30 15:36:00,612 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users JOIN addresses ON users.id = addresses.user_id
2019-07-30 15:36:00,613 INFO sqlalchemy.engine.base.Engine {}


[<User(name=jack, fullname=Jack Bean, nickname=gjffdd),
 <User(name=jack, fullname=Jack Bean, nickname=gjffdd),
 <User(name=jack, fullname=Jack Bean, nickname=gjffdd),
 <User(name=jack, fullname=Jack Bean, nickname=gjffdd)]

In [59]:
session.query(User).join(User.addresses).all()

session.query(User).join(Address, User.addresses).all()

session.query(User).join('addresses').all()

2019-07-30 15:39:15,079 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users JOIN addresses ON users.id = addresses.user_id
2019-07-30 15:39:15,080 INFO sqlalchemy.engine.base.Engine {}


[<User(name=jack, fullname=Jack Bean, nickname=gjffdd),
 <User(name=jack, fullname=Jack Bean, nickname=gjffdd),
 <User(name=jack, fullname=Jack Bean, nickname=gjffdd),
 <User(name=jack, fullname=Jack Bean, nickname=gjffdd)]

2019-07-30 15:39:15,084 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users JOIN addresses ON users.id = addresses.user_id
2019-07-30 15:39:15,085 INFO sqlalchemy.engine.base.Engine {}


[<User(name=jack, fullname=Jack Bean, nickname=gjffdd),
 <User(name=jack, fullname=Jack Bean, nickname=gjffdd),
 <User(name=jack, fullname=Jack Bean, nickname=gjffdd),
 <User(name=jack, fullname=Jack Bean, nickname=gjffdd)]

2019-07-30 15:39:15,089 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users JOIN addresses ON users.id = addresses.user_id
2019-07-30 15:39:15,090 INFO sqlalchemy.engine.base.Engine {}


[<User(name=jack, fullname=Jack Bean, nickname=gjffdd),
 <User(name=jack, fullname=Jack Bean, nickname=gjffdd),
 <User(name=jack, fullname=Jack Bean, nickname=gjffdd),
 <User(name=jack, fullname=Jack Bean, nickname=gjffdd)]

+ join only retrieves Users, where address field is defined
+ for all Users, you can use outerjoin

In [60]:
session.query(User).outerjoin(Address).all()

2019-07-30 15:39:38,540 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users LEFT OUTER JOIN addresses ON users.id = addresses.user_id
2019-07-30 15:39:38,541 INFO sqlalchemy.engine.base.Engine {}


[<User(name=jack, fullname=Jack Bean, nickname=gjffdd),
 <User(name=jack, fullname=Jack Bean, nickname=gjffdd),
 <User(name=jack, fullname=Jack Bean, nickname=gjffdd),
 <User(name=jack, fullname=Jack Bean, nickname=gjffdd),
 <User(name=wendy, fullname=Wendy Williams, nickname=windy),
 <User(name=mary, fullname=Mary Contrary, nickname=mary),
 <User(name=fred, fullname=Fred Flintstone, nickname=freddy),
 <User(name=fred, fullname=Fred Flintstone, nickname=freddy),
 <User(name=wendy, fullname=Wendy Williams, nickname=windy),
 <User(name=fred, fullname=Fred Flintstone, nickname=freddy),
 <User(name=mary, fullname=Mary Contrary, nickname=mary),
 <User(name=wendy, fullname=Wendy Williams, nickname=windy),
 <User(name=mary, fullname=Mary Contrary, nickname=mary),
 <User(name=fred, fullname=Fred Flintstone, nickname=freddy),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=ed, fullname=Ed Jones, nickname=Eddo),
 <User(name=wendy, fullname=Wendy Williams, nickname=windy),
 <User(n

In [61]:
session.query(User, Address).select_from(Address).join(User).all()

2019-07-30 15:40:22,650 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname, addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses JOIN users ON users.id = addresses.user_id
2019-07-30 15:40:22,651 INFO sqlalchemy.engine.base.Engine {}


[(<User(name=jack, fullname=Jack Bean, nickname=gjffdd),
  <Address(email_address=jack@google.com)>),
 (<User(name=jack, fullname=Jack Bean, nickname=gjffdd),
  <Address(email_address=j25@yahoo.com)>),
 (<User(name=jack, fullname=Jack Bean, nickname=gjffdd),
  <Address(email_address=jack@google.com)>),
 (<User(name=jack, fullname=Jack Bean, nickname=gjffdd),
  <Address(email_address=j25@yahoo.com)>),
 (<User(name=jack, fullname=Jack Bean, nickname=gjffdd),
  <Address(email_address=jack@google.com)>),
 (<User(name=jack, fullname=Jack Bean, nickname=gjffdd),
  <Address(email_address=j25@yahoo.com)>),
 (<User(name=jack, fullname=Jack Bean, nickname=gjffdd),
  <Address(email_address=jack@google.com)>),
 (<User(name=jack, fullname=Jack Bean, nickname=gjffdd),
  <Address(email_address=j25@yahoo.com)>)]

### Aliases
+ for several references to same table you can use aliased()
+ then you join the different aliases of same table to retrieve several values

In [62]:
from sqlalchemy.orm import aliased
add_alias1 = aliased(Address)
add_alias2 = aliased(Address)
session.query(User.name, add_alias1.email_address, add_alias2.email_address).\
    join(add_alias1, User.addresses).\
    join(add_alias2, User.addresses).\
    filter(add_alias1.email_address == 'jack@google.com').\
    filter(add_alias2.email_address == 'j25@yahoo.com').all()


2019-07-30 15:41:55,563 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name, addresses_1.email_address AS addresses_1_email_address, addresses_2.email_address AS addresses_2_email_address 
FROM users JOIN addresses AS addresses_1 ON users.id = addresses_1.user_id JOIN addresses AS addresses_2 ON users.id = addresses_2.user_id 
WHERE addresses_1.email_address = %(email_address_1)s AND addresses_2.email_address = %(email_address_2)s
2019-07-30 15:41:55,564 INFO sqlalchemy.engine.base.Engine {'email_address_1': 'jack@google.com', 'email_address_2': 'j25@yahoo.com'}


[('jack', 'jack@google.com', 'j25@yahoo.com'),
 ('jack', 'jack@google.com', 'j25@yahoo.com'),
 ('jack', 'jack@google.com', 'j25@yahoo.com'),
 ('jack', 'jack@google.com', 'j25@yahoo.com')]

### subqueries
+ first create a subquery using .subquery() which returns an SQL expression construct
+ here, we use subquery for an aggregate function count()
+ the subquery can be treated like a standard table with attributes accessible as c.

In [79]:
from sqlalchemy import func
sub_query = session.query(Address.user_id, func.count('*').label('address_count')).\
    group_by(Address.user_id).subquery()
sub_query

<sqlalchemy.sql.selectable.Alias at 0x11d4997b8; %(4786329528 anon)s>

In [82]:
session.query(User, sub_query.c.address_count).\
    outerjoin(sub_query, User.id == sub_query.c.user_id).\
    order_by(User.id).all()

2019-07-31 14:20:04,288 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname, anon_1.address_count AS anon_1_address_count 
FROM users LEFT OUTER JOIN (SELECT addresses.user_id AS user_id, count(%(count_1)s) AS address_count 
FROM addresses GROUP BY addresses.user_id) AS anon_1 ON users.id = anon_1.user_id ORDER BY users.id
2019-07-31 14:20:04,289 INFO sqlalchemy.engine.base.Engine {'count_1': '*'}


[(<User(name=ed, fullname=Ed Jones, nickname=Eddy), None),
 (<User(name=wendy, fullname=Wendy Williams, nickname=windy), None),
 (<User(name=mary, fullname=Mary Contrary, nickname=mary), None),
 (<User(name=fred, fullname=Fred Flintstone, nickname=freddy), None),
 (<User(name=jack, fullname=Jack Bean, nickname=gjffdd), 2),
 (<User(name=ed, fullname=Ed Jones, nickname=Eddo), None),
 (<User(name=wendy, fullname=Wendy Williams, nickname=windy), None),
 (<User(name=mary, fullname=Mary Contrary, nickname=mary), None),
 (<User(name=fred, fullname=Fred Flintstone, nickname=freddy), None),
 (<User(name=jack, fullname=Jack Bean, nickname=gjffdd), 2),
 (<User(name=ed, fullname=Ed Jones, nickname=Eddo), None),
 (<User(name=wendy, fullname=Wendy Williams, nickname=windy), None),
 (<User(name=mary, fullname=Mary Contrary, nickname=mary), None),
 (<User(name=fred, fullname=Fred Flintstone, nickname=freddy), None),
 (<User(name=ed, fullname=Ed Jones, nickname=Eddo), None),
 (<User(name=wendy, fullnam

### Exists

In [ ]:
from sqlalchemy.sql import exists
existing = exists().where(Address.user_id != User.id)
session.query(User.name).filter(existing).all()

+ EXISTS can be expresses as any(condition) in SQLalchemy

In [ ]:
session.query(User.name).filter(~User.addresses.any()).all()

In [ ]:
session.query(User.name).filter(User.addresses.any(Address.email_address.like('%google%'))).all()

### Eager Loading
+ if you want to load associated data in a single transaction, you can use selectinload
+ in selectinload, the related data is loaded in a separate select call
+ first it loads all User rows and looks up all primary keys of these objects
+ the second SELECT uses a WHERE addresses.user_id IN (`all primary keys`)
+ selectinload is the preferred method of loading 

In [65]:
from sqlalchemy.orm import selectinload
session.query(User).options(selectinload(User.addresses)).\
    filter_by(name='jack').first().addresses

2019-07-30 16:01:33,104 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = %(name_1)s 
 LIMIT %(param_1)s
2019-07-30 16:01:33,105 INFO sqlalchemy.engine.base.Engine {'name_1': 'jack', 'param_1': 1}
2019-07-30 16:01:33,108 INFO sqlalchemy.engine.base.Engine SELECT addresses.user_id AS addresses_user_id, addresses.id AS addresses_id, addresses.email_address AS addresses_email_address 
FROM addresses 
WHERE addresses.user_id IN (%(primary_keys_1)s) ORDER BY addresses.user_id, addresses.id
2019-07-30 16:01:33,108 INFO sqlalchemy.engine.base.Engine {'primary_keys_1': 6}


[<Address(email_address=jack@google.com)>,
 <Address(email_address=j25@yahoo.com)>]

+ in joinedload, the data is joined in one select call

In [66]:
from sqlalchemy.orm import joinedload
session.query(User).options(joinedload(User.addresses)).\
    filter_by(name='jack').first().addresses

2019-07-30 16:03:13,892 INFO sqlalchemy.engine.base.Engine SELECT anon_1.users_id AS anon_1_users_id, anon_1.users_name AS anon_1_users_name, anon_1.users_fullname AS anon_1_users_fullname, anon_1.users_nickname AS anon_1_users_nickname, addresses_1.id AS addresses_1_id, addresses_1.email_address AS addresses_1_email_address, addresses_1.user_id AS addresses_1_user_id 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = %(name_1)s 
 LIMIT %(param_1)s) AS anon_1 LEFT OUTER JOIN addresses AS addresses_1 ON anon_1.users_id = addresses_1.user_id ORDER BY addresses_1.id
2019-07-30 16:03:13,893 INFO sqlalchemy.engine.base.Engine {'name_1': 'jack', 'param_1': 1}


[<Address(email_address=jack@google.com)>,
 <Address(email_address=j25@yahoo.com)>]

+ contains_eager preloads the associated data to be used in chained filters
+ in example, Address.user is eager_loaded to allow filtering with User.name

In [ ]:
from sqlalchemy.orm import contains_eager
session.query(Address).join(Address.user).\
    filter(User.name=='jack').\
    options(contains_eager(Address.user)).all()

### Deleting
+ without cascading, data related to the deleted data will still be available

In [ ]:
session.delete(jack)
session.query(Address).filter(Address.email_address.in_(['jack@google.com', 'j25@yahoo.com'])).count()

+ with cascading, linked data is deleted as well
+ here, we reset and declare the User class with cascading

In [ ]:
session.close()
Base = declarative_base()

class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
     
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)
    addresses = relationship("Address", back_populates='user',
        cascade="all, delete, delete-orphan")

    def __repr__(self):
        return "<User(name='%s', fullname='%s', nickname='%s')>" % (
            self.name, self.fullname, self.nickname)
    
    
class Address(Base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return "<Address(email_address='%s')>" % self.email_address
    
Base.metadata.create_all(engine)


+ with cascading activate, all associated data will be removed with the object

In [ ]:
session.delete(jack)
session.query(Address).filter(Address.email_address.in_(['jack@google.com', 'j25@yahoo.com'])).count()

### Many to Many Relationship
+ for many2many, you can use an association table to combine both related tables

In [ ]:
from sqlalchemy import Table, Text
# association table
post_keywords = Table('post_keywords', Base.metadata,\
            Column('post_id', ForeignKey('posts.id'), primary_key=True),\
            Column('keyword_id', ForeignKey('keywords.id'), primary_key=True))

In [ ]:
class BlogPost(Base):
    __tablename__ = 'posts'
    
    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('users.id'))
    headline = Column(String(255), nullable=False)
    body = Column(Text)
    
    # here the relationship via the association table is created
    keywords = relationship('Keyword', secondary=post_keywords, back_populates='posts')
    
    # init is optional in SQLalchemy
    def __init__(self, headline, body, author):
        self.author = author
        self.headline = headline
        self.body = body
        
    def __repr__(self):
        return f"BlogPost({self.headline}, {self.body}, {self.author})"
    
class Keyword(Base):
    __tablename__ = 'keywords'
    
    id = Column(Integer, primary_key=True)
    keyword = Column(String(50), nullable=False, unique=True)
    posts = relationship('BlogPost', secondary=post_keywords, back_populates='keywords')
    
    # init is optional in SQLalchemy
    def __init__(self, keyword):
        self.keyword = keyword
    

In [ ]:
BlogPost.author = relationship(User, back_populates='posts')
User.posts = relationship(BlogPost, back_populates="author", lazy="dynamic")

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
wendy = session.query(User).filter_by(name='wendy').first()
post = BlogPost("Wendy's Blog Post", "This is a test", wendy)
session.add(post)

In [ ]:
post.keywords.append(Keyword('wendy'))
post.keywords.append(Keyword('firstpost'))

In [ ]:
session.commit()

In [ ]:
session.query(BlogPost).filter(BlogPost.keywords.any(keyword='firstpost')).all()

In [ ]:
session.query(BlogPost).filter(BlogPost.author==wendy).\
filter(BlogPost.keywords.any(keyword='firstpost')).all()

# Pandas with SQLAlchemy

In [ ]:
data = pd.read_sql_table('users', engine)

In [ ]:
data

In [ ]:
for d in data:
    print(d)